# 📄 Criação da Tabela `SilverYellow`
Este script unifica os dados mensais de viagens **Yellow Taxi** de diferentes meses (`Mes01` a `Mes05`)  
em uma única tabela `SilverYellow`, utilizando a operação `UNION` removendo os registros duplicados.

---

## 🎯 Objetivo
Consolidar dados de múltiplas tabelas mensais em uma única base para facilitar análises, relatórios e KPIs globais.

---


In [0]:
%sql

-- Seleciona o catálogo e schema de destino
USE CATALOG ifood_case;
USE SCHEMA  nytaxi;

-- Cria ou substitui a tabela unificada BronzeYellow
CREATE OR REPLACE TABLE SilverYellow AS

-- =========================================
-- 1️⃣ União de todas as tabelas mensais
--    - Tabelas de origem BronzeYellowMesXX já estão tratadas e higienizadas
--    - Remove duplicatas
-- =========================================
SELECT * FROM BronzeYellowMes01
UNION 
SELECT * FROM BronzeYellowMes02
UNION 
SELECT * FROM BronzeYellowMes03
UNION 
SELECT * FROM BronzeYellowMes04
UNION
SELECT * FROM BronzeYellowMes05;


# ✅ Validação da tabela `SilverYellow`

Nesta versão:
- Cada regra de validação gera um elemento no array `lista_alertas`
- Valores `NULL` são removidos do array de mensagens
- `num_alertas`: quantidade de alertas por registro
- `alerta_validacao`: string única com alertas separados por `"; "`

---


In [0]:
%sql
WITH SilverYellowValidate AS (
  SELECT
   *
    -- =========================================
    -- ARRAY DE REGRAS DE VALIDAÇÃO
    -- Cada WHEN verdadeiro gera mensagem; falso gera NULL
    -- =========================================
    , array_remove(ARRAY(
      CASE WHEN Data_Fim_Viagem IS NULL
            OR (Data_Fim_Viagem < "2023-01-01" OR Data_Fim_Viagem > "2023-05-31")
            THEN 'data fim viagem inválido' END

      , CASE WHEN Data_Inicio_Viagem IS NULL
            OR (Data_Inicio_Viagem < "2023-01-01" OR Data_Inicio_Viagem > "2023-05-31")
            THEN 'data início viagem inválido' END

      , CASE WHEN Dia_Fim_Viagem IS NULL OR Dia_Fim_Viagem < 1 OR Dia_Fim_Viagem > 31
            THEN 'Dia_Fim_Viagem fora do intervalo' END

      , CASE WHEN Dia_Inicio_Viagem IS NULL OR Dia_Inicio_Viagem < 1 OR Dia_Inicio_Viagem > 31
            THEN 'Dia_Inicio_Viagem fora do intervalo' END

      , CASE WHEN Hora_Fim_Viagem IS NULL OR Hora_Fim_Viagem < '00' OR Hora_Fim_Viagem > '23'
            THEN 'Hora_Fim_Viagem fora do intervalo' END

      , CASE WHEN Hora_Inicio_Viagem IS NULL OR Hora_Inicio_Viagem < '00' OR Hora_Inicio_Viagem > '23'
            THEN 'Hora_Inicio_Viagem fora do intervalo' END

      , CASE WHEN Mes_Fim_Viagem IS NULL OR Mes_Fim_Viagem < 1 OR Mes_Fim_Viagem > 5
            THEN 'Mes_Fim_Viagem fora do intervalo' END

      , CASE WHEN Mes_Inicio_Viagem IS NULL OR Mes_Inicio_Viagem < 1 OR Mes_Inicio_Viagem > 5
            THEN 'Mes_Inicio_Viagem fora do intervalo' END

      , CASE WHEN passenger_count IS NULL OR passenger_count <= 0
            THEN 'passenger_count inválido' END

      , CASE WHEN total_amount IS NULL OR total_amount <= 0
            THEN 'total_amount inválido' END

      , CASE WHEN vendorId IS NULL OR (vendorId NOT IN (1, 2, 6, 7))
            THEN 'vendorId inválido' END
    ), NULL) AS lista_alertas
  FROM BronzeYellow
)

SELECT
    *
    , SIZE(lista_alertas)             AS  num_alertas       -- Quantidade de alertas
    , CONCAT_WS('; ', lista_alertas)  AS  alerta_validacao  -- String com alertas separados
FROM 
      SilverYellowValidate
WHERE 
  SIZE(lista_alertas) > 0; -- Retorna somente registros com alertas
